In [1]:
import os

os.environ['KERAS_BACKEND'] = "torch"

# Dataset Generation
Our dataset is a corpus of reviews scrapped from the BGG API. <br /> 
In order to download the comments we make use of the ```bgg_corpus_service.py``` content.

BGG Does not directly provide a way to list all the games it has in archive therefore we used a dump created by the community (2024-08-18).

In [ ]:
# /resources/2024-08-18.csv

# Custom Dataloaders Definitions
To train the model we require a way to get elements of our dataset. I designed 3 classes for this:

In [2]:
import dataset

lazy = dataset.LazyCommentDataset("../data/corpus.preprocessed.csv")
dt = dataset.CommentDataset("../data/corpus.preprocessed.csv")

Index        15522432
comments    379211268
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 1940304 entries, 0 to 1940425
Data columns (total 1 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   comments  object
dtypes: object(1)
memory usage: 376.4 MB
None


Without preprocessing on dataset: 379211268 <br />
If the dataset is processed already: 677671688 (Twice the size) <br />

In [3]:
from torch.utils.data import DataLoader

dt_dataloader = DataLoader(dt, batch_size=32, shuffle=True, collate_fn=lambda x: x)
lazy_dataloader = DataLoader(lazy, batch_size=32, shuffle=True, collate_fn=lambda x: x)

In [4]:
dt_iterator = iter(dt_dataloader)
lazy_iterator = iter(lazy_dataloader)